In [1]:
from NTracker.parsers.coco_results_parser import CocoResultsParser
from NTracker.features_generators import FeaturesLoader, OpencvPredictor, TorchreidPredictor
from pathlib import Path
import numpy as np

/home/egracia/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
parser = CocoResultsParser(Path("/home/egracia/Desktop/pigs/predict"))

In [3]:
instances = parser.read(Path("/home/egracia/Desktop/pigs/predict/10000.json"))

In [4]:
print(instances)
for i in instances:
    print(i)

[<NTracker.utils.structures.Instance object at 0x7f18f806d250>, <NTracker.utils.structures.Instance object at 0x7f1895412a30>, <NTracker.utils.structures.Instance object at 0x7f192028c790>, <NTracker.utils.structures.Instance object at 0x7f192028c9d0>, <NTracker.utils.structures.Instance object at 0x7f192028ca90>, <NTracker.utils.structures.Instance object at 0x7f192028c7f0>, <NTracker.utils.structures.Instance object at 0x7f192028ce50>, <NTracker.utils.structures.Instance object at 0x7f192028c9a0>, <NTracker.utils.structures.Instance object at 0x7f192028c7c0>, <NTracker.utils.structures.Instance object at 0x7f191c1724f0>]
{'bounding_box': (1286, 382, 1529, 498), 'score': 1.0, 'id': 0, 'mask': array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
     

In [5]:
loader = FeaturesLoader(
    "/home/egracia/Desktop/pigs_tracking/predict/torchreid",
    True,
    True
)
f = loader.predict(Path("10000.png"), 0)
print(f, f.shape)
b = loader.predict_batch(Path("10000.png"))
print(b, b.shape)
np.testing.assert_array_equal(f, b[0])

b.reshape((b.shape[0], -1)).shape


[4.22162712e-02 2.45056581e-02 6.31410658e-01 0.00000000e+00
 0.00000000e+00 2.62668192e-01 7.55686581e-01 5.54146111e-01
 1.78241897e+00 0.00000000e+00 1.77144229e-01 0.00000000e+00
 0.00000000e+00 7.43201375e-01 0.00000000e+00 0.00000000e+00
 0.00000000e+00 6.76705003e-01 0.00000000e+00 0.00000000e+00
 1.67323992e-01 1.58006877e-01 0.00000000e+00 4.26226854e-02
 6.38445139e-01 0.00000000e+00 9.36980009e-01 1.04678996e-01
 0.00000000e+00 0.00000000e+00 0.00000000e+00 5.41549027e-01
 4.64811444e-01 2.63773296e-02 0.00000000e+00 1.58954456e-01
 5.70768654e-01 5.41569293e-01 0.00000000e+00 4.84329879e-01
 6.66959882e-01 9.16767597e-01 0.00000000e+00 0.00000000e+00
 5.68500757e-01 0.00000000e+00 6.41107380e-01 4.04952615e-01
 6.76554739e-01 0.00000000e+00 5.09352505e-01 5.41916847e-01
 0.00000000e+00 1.21060634e+00 0.00000000e+00 4.02527779e-01
 1.83173621e+00 7.85834640e-02 9.17651691e-03 0.00000000e+00
 5.87269127e-01 3.41884792e-02 2.33475581e-01 3.26904356e-01
 8.50397468e-01 0.000000

(10, 512)

In [6]:
img = np.ones((1080,1920,3), dtype="uint8")
pred = OpencvPredictor(
    model_path="/home/egracia/GIT/NTracker/models/resnet101.pb",
    image_size=224,
    norm_mean=[103.939, 116.779, 123.68],
    norm_std=[1., 1., 1.],
    channel_first=True,
    mean_output=True,
    flatten_output=False,
    color_mode="RGB",
    device="CPU"
)

pred = TorchreidPredictor(
    model_name="osnet_x1_0",
    model_path="/home/egracia/GIT/NTracker/models/osnet_x1_0_market_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip.pth",
    image_size=[256, 128],
    norm_mean=[0.485, 0.456, 0.406],
    norm_std=[0.229, 0.224, 0.225],
    device="CPU",
)


p = pred.predict(img)
print(p.shape)
b = pred.predict_batch([img])
print(b.shape)



Model: osnet_x1_0
- params: 2,193,616
- flops: 978,878,352
Successfully loaded pretrained weights from "/home/egracia/GIT/NTracker/models/osnet_x1_0_market_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
torch.Size([512])
(512,)
torch.Size([1, 512])
(1, 512)
